Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:

import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from icecream import ic

Cost: $d + (d \cdot \alpha \cdot w)^\beta$ with $\alpha \ge 0$ and $\beta \ge 0$

## Genetico

In [51]:
# from importlib.resources import path
# import numpy as np
# import networkx as nx
# import random

# class GA_Solver:

#     def __init__(self, problem, pop_size=50, generations=100, offprint=20):
#         self.prob = problem
#         self.graph = problem.graph
#         # Tutte le città tranne il deposito
#         self.cities_to_visit = [n for n in self.graph.nodes if n != 0]
#         # Precalcolo cammini minimi
#         self.all_paths = dict(nx.all_pairs_dijkstra_path(self.graph, weight='dist'))
#         self.dist_matrix = dict(nx.all_pairs_dijkstra_path_length(self.graph, weight='dist'))
#         self.pop_size = pop_size
#         self.generations = generations
#         self.offprint = offprint

#     def _calc_path_cost_constat_weight(self, path : list[int], weight: float) -> float:
#         c = 0
#         current_w = weight
#         for u, v in zip(path, path[1:]):
#             d = self.dist_matrix[u][v]
#             # Formula del costo: d + (alpha * d * w)^beta
#             c += d + (self.prob.alpha * d * current_w) ** self.prob.beta
#         return c

#     def _path_cost(self, path: list[tuple[int, float]]) -> float:
#         total_cost = 0
#         current_weight = 0
#         for i in range(len(path)-1):
#             u, v = path[i][0], path[i+1][0]
#             d = self.graph[u][v]['dist']
#             total_cost += d + (self.prob.alpha * d * current_weight) ** self.prob.beta
#             if v==0:
#                 current_weight = 0  # Svuota il peso al deposito
#             else:
#                 current_weight += path[i+1][1]  # aggiungo oro raccolto al peso
#         return total_cost  
    
#     def greedy_initialization(self)-> tuple[list[tuple[int, float]], float]:
#         """Costruisce un individuo iniziale usando una strategia greedy."""

#         if self.prob.beta >= 1.0 :
#             # Se beta è alto, il costo del peso è molto penalizzante, quindi conviene fare più viaggi con carichi leggeri
#             # potrebbe convenire addirittura portare poco oro per volta (prendere solo una parte dell'oro disponibile in ogni città) e fare più giri
#             #print(f"Greedy initialization con beta alto: favorisco più viaggi con carichi leggeri. {self.prob.beta}")
#             ind = list(self.cities_to_visit)
#             random.shuffle(ind)
#             chromo, cost= self._evaluate_and_segment(ind)
#             chromo, cost= self.multiple_cycle(chromo)

#         else:
#             # Se beta è basso, il costo del peso è meno penalizzante, quindi conviene fare meno viaggi con carichi più pesanti:
#             ind = list(self.cities_to_visit)
#             random.shuffle(ind)
#             chromo, cost= self._evaluate_and_segment(ind)

#         return chromo, cost
    
    # def multiple_cycle (self, chromo: list[tuple[int, float]])-> tuple[list[tuple[int, float]], float]:
    #     tours = []
    #     current_tour_cities = []
    #     #print(chromo)  
    #     # 1. Dividiamo il percorso originale in tour che partono e tornano a zero (escludendo gli 0)
    #     for n, w in chromo:
    #         if n == 0:
    #             if current_tour_cities:
    #                 #print("Tour trovato:", current_tour_cities)
    #                 tours.append(current_tour_cities)
    #                 current_tour_cities = []
    #         else:
    #             current_tour_cities.append((n,w))
        
    #     trips=[]

    #     for t in tours:
    #       closed_tour= [(0,0)]+ t+ [(0,0)]
    #       cost= self._path_cost( closed_tour)
    #       #print(closed_tour)
    #       # simulo il costo di fare più giri in uno stesso tour
    #       #print(f"costo tour {cost}")
    #       set_t= [ (c,w) for c,w in t if c!=0 and w!=0]
          
    #       best_trip=closed_tour

    #       for i in range(2, int (min(set_t, key=lambda x: x[1])[1] ) ):
    #           #print(f"Divido il carico per {i}")
    #           trip=[(0,0)]
    #           for j in range(i):
    #             # raccogliere tutto l'oro capire come fare se non è divisibile 
    #             r= [ 0 for _ in t]
    #             if j== i-1:
    #                 r= [ w % i for c, w in t] # se non è divisibile, aggiungo il resto all'ultimo giro
    #             trip.extend( [ (c, w//i + r) for (c, w), r in zip(t, r) ]) 
    #             trip.extend([(0,0)])
    #           cost_trip= self._path_cost( trip)       
    #           if cost_trip < cost and cost_trip>0:
    #               cost= cost_trip
    #               best_trip= trip
    #           else:
    #               break
    #       #print (f"Best trip per questo tour: {best_trip} con costo {cost}")
    #       trips.extend(best_trip[:-1])  # escludiamo l'ultimo 0 che sarà aggiunto dal tour successivo
        
    #     trips.append((0,0))  # aggiungiamo l'ultimo ritorno al deposito alla fine di tutti i tour
    #     #print(f"Trips dopo ottimizzazione multi-cycle: {trips}")
    #     total_cost= self._path_cost( trips) 
    #     #print(f"Costo totale dopo ottimizzazione multi-cycle: {total_cost}")
    #     return trips, total_cost



#     def _evaluate_and_segment(self, chromosome: list[int]) -> tuple[list[tuple[int, float]], float]:
#         """Decoder Greedy: decide se tornare al deposito per scaricare."""
#         route = []
#         total_cost = 0
#         current_node = 0
#         current_weight = 0

#         route.append((0, 0)) 
#         for next_target in chromosome:
#             # Opzione A: Diretto
#             path_direct = self.all_paths[current_node][next_target]
#             cost_direct = self._calc_path_cost_constat_weight(path_direct, current_weight)
            
#             # Opzione B: Passando dal deposito
#             path_to_depot = self.all_paths[current_node][0]
#             path_from_depot = self.all_paths[0][next_target]
#             cost_unload = (self._calc_path_cost_constat_weight(path_to_depot, current_weight) + 
#                            self._calc_path_cost_constat_weight(path_from_depot, 0))
            
#             if current_weight > 0 and cost_unload < cost_direct:
#                 # Scarico al deposito
#                 for v in path_to_depot[1:]:
#                     route.append((v, 0))
#                 current_weight = 0
#                 # Vado al target
#                 for v in path_from_depot[1:]:
#                     gold = self.graph.nodes[v].get('gold', 0) if v == next_target else 0
#                     route.append((v, gold))
#                     current_weight += gold
#                 total_cost += cost_unload
#             else:
#                 # Vado diretto
#                 for v in path_direct[1:]:
#                     gold = self.graph.nodes[v].get('gold', 0) if v == next_target else 0
#                     route.append((v, gold))
#                     current_weight += gold
#                 total_cost += cost_direct
#             current_node = next_target

#         # Ritorno finale
#         path_home = self.all_paths[current_node][0]
#         total_cost += self._calc_path_cost_constat_weight(path_home, current_weight)
#         for node in path_home[1:]:
#             route.append((node, 0))
            
#         return route, total_cost
    
#     def _optimize_baseline(self, chromosome: list[int]) -> tuple[list[tuple[int, float]], float]:
#         n = len(chromosome)
#         # V[i] = miglior costo per coprire le prime i città del cromosoma
#         V = [float('inf')] * (n + 1)
#         V[0] = 0
#         # p[i] = l'indice della città precedente da cui siamo tornati al deposito
#         p = [0] * (n + 1)

#         # Trasformiamo il cromosoma in una sequenza 1-based per comodità
#         nodes = [0] + chromosome 

#         for i in range(1, n + 1):
#             current_weight = 0
#             current_dist_cost = 0
            
#             # Proviamo a creare un "tour" che finisce alla città i
#             # partendo dal deposito dopo aver visitato la città j
#             for j in range(i - 1, -1, -1):
#                 # Calcoliamo il costo del tour: 0 -> nodes[j+1] -> ... -> nodes[i] -> 0
                
#                 # 1. Costo da deposito a prima città del segmento
#                 segment_cost = self._calc_path_cost_constat_weight(self.all_paths[0][nodes[j+1]], 0)
                
#                 # 2. Costo interno al segmento
#                 w = 0
#                 for k in range(j + 1, i):
#                     w += self.graph.nodes[nodes[k]]['gold']
#                     path = self.all_paths[nodes[k]][nodes[k+1]]
#                     segment_cost += self._calc_path_cost_constat_weight(path, w)
                
#                 # 3. Costo ritorno al deposito dall'ultima città del segmento
#                 w += self.graph.nodes[nodes[i]]['gold']
#                 segment_cost += self._calc_path_cost_constat_weight(self.all_paths[nodes[i]][0], w)

#                 # Se usare questo segmento è meglio della soluzione precedente, aggiorna
#                 if V[j] + segment_cost < V[i]:
#                     V[i] = V[j] + segment_cost
#                     p[i] = j
                
#                 # (Opzionale) Se il peso diventa assurdo, inutile continuare a unire città
#                 if w > 10000: # Inserisci un limite logico se necessario
#                     break

#         # Ricostruzione della route (usando p)
#         final_route = [(0, 0)]
#         curr = n
#         segments = []
#         while curr > 0:
#             segments.append((p[curr], curr))
#             curr = p[curr]
#         segments.reverse()

#         # Trasforma i segmenti nel formato (nodo, oro) richiesto
#         for start, end in segments:
#             # Da 0 alla prima città del segmento
#             path_to_first = self.all_paths[0][nodes[start+1]]
#             for v in path_to_first[1:]:
#                 gold = self.graph.nodes[v]['gold'] if v == nodes[start+1] else 0
#                 final_route.append((v, gold))
            
#             # Tra le città del segmento
#             for k in range(start + 1, end):
#                 path = self.all_paths[nodes[k]][nodes[k+1]]
#                 for v in path[1:]:
#                     gold = self.graph.nodes[v]['gold'] if v == nodes[k+1] else 0
#                     final_route.append((v, gold))
            
#             # Ritorno al deposito
#             path_home = self.all_paths[nodes[end]][0]
#             for v in path_home[1:]:
#                 final_route.append((v, 0))

#         return final_route, V[n]

#     # def _create_individual(self) -> tuple[list[tuple[int, float]], float]:
#     #     ind = list(self.cities_to_visit)
#     #     random.shuffle(ind)
#     #     return self._evaluate_and_segment(ind)
    
#     def _mutate_merge_tours(self, route: list[tuple[int, float]]) -> tuple[list[tuple[int, float]], float]:
#         # 1. Estraiamo i tour come liste di città (escludendo gli zeri)
#         tours = []
#         current_tour = []
#         for n, w in route:
#             if n == 0:
#                 if current_tour:
#                     tours.append(current_tour)
#                     current_tour = []
#             else:
    
#                 current_tour.append((n,w))
        
#         if len(tours) < 2:
#             return route, self._path_cost(route)

#         # 2. Proviamo a fondere due tour casuali consecutivi
#         #print(f"Tour attuali: {tours}")
#         idx = random.randint(0, len(tours) - 2)
#         tour_a = tours[idx]
#         tour_b = tours[idx+1]
#         #print(f"Provo a fondere tour {idx} e {idx+1}, Tour A: {tour_a}, Tour B: {tour_b}")
        
#         # Calcoliamo il costo dei due tour separati (con ritorno al deposito)
#         # tour_a: 0 -> ... -> 0  e tour_b: 0 -> ... -> 0
#         cost_separate = self._path_cost([(0, 0)] +  tour_a + [(0, 0)]) + \
#                         self._path_cost([(0, 0)] + tour_b + [(0, 0)])
        
#         #print(f"Costo tour separati: {cost_separate}")
#         # Proviamo a unirli: 0 -> tour_a -> tour_b -> 0
#         city_gold_map = {}
#         for c, w in tour_a + tour_b:
#             if w > 0:
#                 city_gold_map[c] = w
        
#         if not city_gold_map:
#             return route, self._path_cost(route)
        
#         target_cities = list(city_gold_map.keys())
        
#         #print(f"Città da visitare per il tour unificato: {target_cities}")
#         # find the TSP tour that visits all target cities
#         # prima la città più distante da 0 poi le altr
        
#         # --- LOGICA TSP STRATEGICO ---
#         merged_tour = []
        
#         # 1. Troviamo la città nel cluster che è più LONTANA dal deposito
#         far_city = max(target_cities, key=lambda c: self.dist_matrix[0][c])
        
#         # ma assicuriamoci che l'ordine tenda a "chiudere" verso il deposito.
#         current_node = far_city
        
#         merged_tour.extend(self.all_paths[0][current_node][:-1])  # percorso dal deposito alla città più lontana
#         target_cities.remove(current_node)
#         #print(f"Parto da città più lontana: {current_node}, percorso iniziale: {merged_tour}")
        
#         while target_cities:
#             # Tra le rimaste, scegliamo la più vicina alla posizione attuale
#             next_node = min(target_cities, key=lambda c: self.dist_matrix[current_node][c])
#             merged_tour.extend(self.all_paths[current_node][next_node][:-1])  # percorso tra current_node e next_node
#             target_cities.remove(next_node)
#             current_node = next_node
        
#         merged_tour.extend(self.all_paths[current_node][0])  # percorso dalla città finale al deposito
#         #print(f"Tour unificato: {merged_tour}")
        
#         # aggiungo i pesi corretti al tour unificato
#         merged_tour_with_weights = [ ]
#         for c in merged_tour:
#             w= city_gold_map[c] if c in city_gold_map else 0
#             merged_tour_with_weights.append((c,w))
#         #print(f"Tour unificato con pesi: {merged_tour_with_weights}")
#         cost_merged = self._path_cost( merged_tour_with_weights)   
#         #print(f"Costo tour unificato: {cost_merged}")
        
#         # 3. Se l'unione è vantaggiosa, aggiorniamo la lista dei tour
#         if cost_merged < cost_separate:
#             tours[idx] = merged_tour_with_weights[1:-1]  # aggiorniamo il tour a idx con il nuovo tour unito (escludendo i depositi)
#             tours.pop(idx + 1)

#         #print(f"Cost separate: {cost_separate}, Cost merged: {cost_merged}, {'Merged' if cost_merged < cost_separate else 'Kept separate'}")
#         #print(f"Tour A: {tour_a}, Tour B: {tour_b}, Merged Tour: {merged_tour}")
#         # 4. Ricostruiamo il cromosoma finale unendo tutti i tour rimasti
#         # Usiamo lo Split su tutto per essere sicuri della segmentazione ottima
#         final_route = [(0, 0)]
#         for tour in tours:
#             final_route.extend(tour)
#             final_route.append((0, 0))
#         #print(f"Cromosoma finale dopo mutazione merge: {final_route}")  
#         return final_route, self._path_cost(final_route)

#     def _mutate(self, ind: list[tuple[int, float]] ) -> tuple[list[tuple[int, float]], float]:
#         """
#         Mutation: 2 possibilità 
#         1. Aggregazione di 2 tours: Sceglie 2 tour consecutivi e prova a unirli in un unico tour, valutando se è vantaggioso.
#         2. Ottimizzazione locale: Scompone il percorso in tour (segmenti tra i depositi) e li ri-decodifica (li divide se vantaggioso ulteriormente).
#         """
        
#         # --- CALCOLO THRESHOLD DINAMICA ---
#         # Partiamo da una base di 0.5
#         # Se beta < 1 (distanza dominante), vogliamo threshold alta -> Opzione 1 (Merge)
#         # Se beta > 1 (peso dominante), vogliamo threshold bassa -> Opzione 2 (Split/Optimize)
        
#         # Utilizziamo una funzione sigmoidea o lineare limitata per beta
#         # Se beta è piccolo (es. 0.5), beta_factor è positivo
#         # Se beta è grande (es. 2.0), beta_factor è negativo
#         beta_factor = 0.5 * (1.0 - self.prob.beta)
        
#         # Se beta è vicino a 1, l'influenza di alpha diventa cruciale
#         # Se alpha è alto, il peso "costa" molto -> favoriamo ottimizzazione locale (abbassiamo threshold)
#         # Se alpha è basso, il peso conta poco -> favoriamo aggregazione (alziamo threshold)
#         # Normalizziamo alpha (assumendo che alpha sia solitamente in un range 0.001 - 0.1 o simile)
#         # Usiamo una costante di sensibilità (es. 10.0) per dare peso ad alpha nel caso beta ~= 1
#         alpha_influence = 0.2 * (0.05 - self.prob.alpha) if 0.9 <= self.prob.beta <= 1.1 else 0
        
#         # Calcolo finale con clipping tra 0.1 e 0.9 per garantire un minimo di variabilità
#         threshold = 0.5 + beta_factor + alpha_influence
#         threshold = max(0.1, min(0.9, threshold))

#         ratio = random.random()

#         threshold = 0.5 + 0.8 * (1 - self.prob.beta)

#         if ratio < threshold:
#             # OPZIONE 1:
#             # Swap mutation
#             # chromo = set([n for n, g in ind if n != 0])
#             # chromo= list(chromo)  # rimuoviamo eventuali duplicati
#             # # Applichiamo uno swap casuale per diversità
#             # if len(chromo) >= 2:
#             #     i, j = random.sample(range(len(chromo)), 2)
#             #     chromo[i], chromo[j] = chromo[j], chromo[i]
            
#             # chromo, cost= self._evaluate_and_segment(chromo)
#             # if self.prob.beta >= 1.0:
#             #     chromo, cost= self.multiple_cycle(chromo)
#             chromo, cost= self._mutate_merge_tours(ind)
#             return chromo, cost
        
#         else:
#             # OPZIONE 2:
#             # Ottimizzazione locale: ri-decodifica dei tour
#             tours = []
#             current_tour_cities = []
                
#             # 1. Dividiamo il percorso originale in tour che partono e tornano a zero (escludendo gli 0)
#             for n, w in ind:
#                 if n == 0:
#                     if current_tour_cities:
#                         #print("Tour trovato:", current_tour_cities)
#                         tours.append(current_tour_cities)
#                         current_tour_cities = []
#                 else:
#                     current_tour_cities.append(n)
                        
#             # Aggiungiamo l'ultimo tour se il percorso non finiva con 0
#             if current_tour_cities:
#                 #print("Tour trovato:", current_tour_cities)
#                 tours.append(current_tour_cities)

#             #print("Eseguo ottimizzazione locale sui tour")
#             mutated_ind = []
#             mutated_ind.append((0, 0))
#             # Il percorso deve iniziare con il deposito
#             golden_city= set()
#             # 2. Per ogni tour trovato, proviamo a ri-eseguire il decoder greedy
#             for t in tours:
#                 # t è una lista di soli nomi di città (interi)
#                 # escludiamo le città dove ho già raccolto l'oro
#                 t = [city for city in t if city not in golden_city]
#                 # trasformiamo t in un set
#                 t_set = set(t)
#                 #print ("Tour da ri-decodificare (escludendo città già raccolte):", t_set)
#                 if len(t_set) > 0:
#                     # Re-decodifichiamo il segmento di città
#                     new_segment_route, _ = self._evaluate_and_segment(t_set)
#                     #print("Nuovo segmento dopo ri-decodifica:", new_segment_route)
#                     mutated_ind.extend(new_segment_route[1:])  # escludiamo il primo 0 del segmento, già presente
                    
#                     # Aggiorniamo l'insieme delle città dove abbiamo raccolto l'oro
#                     golden_city.update(t_set)

#             if self.prob.beta >= 1.0:
#                 mutated_ind, cost= self.multiple_cycle(mutated_ind)
#             else:
#                 cost= self._path_cost(mutated_ind)
                
#             return mutated_ind, cost
    
#     def _optimize_tour(self, tour: list[tuple[int, float]]) -> list[tuple[int, float]]:
#         # Ottimizzazione locale per un singolo tour (escluso il deposito)
#         target_city = [(c, w) for c, w in tour if c != 0 and w > 0]
        
#         # Creare un dizionario per mappare città -> oro originale
#         city_to_gold = {c: self.graph.nodes[c]['gold'] for c, w in target_city}
        
#         # Controllo se esiste un percorso più breve per raccogliere l'oro in queste città
#         optimize_tour, _ = self._evaluate_and_segment([c for c, w in target_city])
        
#         new_tour = []
#         for c, w in optimize_tour:
#             # Usa il dizionario per ottenere l'oro originale della città
#             real_w = city_to_gold.get(c, 0)
#             if real_w:
#                 new_tour.append((c, real_w))
#             else:
#                 new_tour.append((c, 0))
        
#         return new_tour

    
#     def _crossover(self, parent1: list[tuple[int, float]], parent2: list[tuple[int, float]]) -> tuple[list[tuple[int, float]], float]:
#         # Crossover: 
#         # estraggo dal primo genitore i tour che completano il raccoglimento di oro per le prime k= tot city/ 2 città e le restanti estraggo i tour dal genitore 2
        
#         k= len(self.cities_to_visit) // 2
#         tours_p1= []
#         collect_gold_in_p1 = {} 
#         tour=[]
#         # ciclo che estrae dal primo genitore i tour che completano il raccoglimento di oro per le prime k città e le restanti estraggo dal genitore 2
        
#         for c, w in parent1[1:]: 
#             if c==0:
#                 # città deposito tour concluso
#                 if tour:
#                     tours_p1.extend([(0,0)]+tour)
#                     tour=[]
#                 # esco dal loop quando quando ho raccolto l'oro in tutte le k città
#                 if len(collect_gold_in_p1) >= k and all(v == 0 for v in collect_gold_in_p1.values()):
#                     break
#             else:
#                 # se la cittè c non è ancora stata visitata e c'è ancora spazio (meno di k città) la inserisco in collect_gold_in_p1
#                 if c not in collect_gold_in_p1 and w>0 and len(collect_gold_in_p1)<k:
#                     # inserisco tuple (c, oro residuo) in collect_gold_in_p1
#                     collect_gold_in_p1[c] = self.graph.nodes[c]['gold']-w 
#                     tour.append((c,w)) 
#                 else:
#                     # sottrarre il valore w al valore già presente in collect_gold_in_p1 per la città c
#                     if c in collect_gold_in_p1:
#                         collect_gold_in_p1[c] = max(0, collect_gold_in_p1[c] - w)
#                         tour.append((c,w))
#                     else:
#                         tour.append((c,0))
#         #print ("Città raccolte da p1:", collect_gold_in_p1)
#         #print("Tour estratti dal genitore 1:", tours_p1)
#         # estraggo i tour dal genitore 2 che completano il raccoglimento di oro per le città non ancora raccolte da p1
#         tours_p2= []
#         # le città che deve raccogliere p2 sono quelle rimanenti 
#         collect_gold_in_p2 = {c: self.graph.nodes[c]['gold'] 
#                           for c in self.graph.nodes 
#                           if c not in collect_gold_in_p1 and c != 0}
#         tour=[]
        
#         for c, w in parent2[1:]:
#             if c==0:
#                 # città deposito tour concluso
#                 if tour:
#                     # ottimizzare il tour t2 prima di aggiungerlo a tours p2
#                     tour = self._optimize_tour(tour) 
#                     tours_p2.extend(tour[:-1]) # escludiamo l'ultimo 0 che sarà aggiunto dal tour successivo
#                     tour=[]
#                 # esco dal loop quando quando ho raccolto tutto l'oro
#                 if all(v == 0 for v in collect_gold_in_p2.values()):
#                     break
#             else:
#                 # se la cittè c non è ancora stata visitata e c'è ancora spazio (meno di k città) la inserisco in collect_gold_in_p2
#                 if c in collect_gold_in_p2 and w>0 :
#                     #print(f"Raccolgo oro in città {c} da genitore 2, oro residuo: {collect_gold_in_p2[c] - w}")
#                     collect_gold_in_p2[c] = max(0, collect_gold_in_p2[c] - w)
#                     tour.append((c,w))
#                 else:
#                     tour.append((c,0))
        
#         offspring = tours_p1 + tours_p2 + [(0,0)]  # aggiungo il ritorno finale al deposito
#         cost = self._path_cost(offspring)

#         return offspring, cost
    
#     def _create_baseline_individual(self) -> tuple[list[tuple[int, float]], float]:
#         # Creo un individuo basato sulla soluzione baseline ogni città target si torna al deposito
#         path= []
#         for target_city in self.cities_to_visit:
#             if self.graph.nodes[target_city]['gold'] == 0:
#                 continue  # se non c'è oro, salto la città
#             andata= self.all_paths[0][target_city]  # percorso dal deposito alla città target
#             path.extend([(c, 0) for c in andata[:-1]])  # aggiungo il percorso di andata (escludendo la città target)
#             w= self.graph.nodes[target_city]['gold']
#             path.append((target_city, w))  # aggiungo la città target con l'oro raccolto
#             ritorno= self.all_paths[target_city][0]  # percorso di ritorno al deposito
#             path.extend([(c, 0) for c in ritorno[1:-1]])  # aggiungo il percorso di ritorno (escludendo city target e deposito)  
            
#         path.extend([(0, 0)])  # aggiungo il ritorno finale al deposito
#         cost= self._path_cost(path)   
#         return path, cost
    
#     def _create_nearest_neighbor_individual(self):
#         unvisited = set(self.cities_to_visit)
#         current_node = 0
#         sequence = []
        
#         while unvisited:
#             # Trova la città più vicina tra quelle rimaste
#             next_city = min(unvisited, key=lambda city: self.dist_matrix[current_node][city])
#             sequence.append(next_city)
#             unvisited.remove(next_city)
#             current_node = next_city
            
#         # Passa questa sequenza al tuo decoder segmentato
#         return self._optimize_baseline(sequence)
        
    
#     def _run_ga_logic(self)-> list[tuple[int, float]]:
#         population = [self.greedy_initialization() for _ in range(self.pop_size-1)]
#         population.append(self.improved_baseline_ind)  
#         #print("Popolazione iniziale creata.")
#         #print(  [cost for cromo, cost in population] )
#         # ordina la popolazione in base al costo (decrescnente)
#         population = sorted(population, key=lambda x: x[1])
#         best_chromo = population[0][0]
#         #print(best_chromo)

#         best_cost = population[0][1]
#         #print(f"Best cost iniziale: {best_cost}")
#         stagnation_counter = 0

#         for gen in range(self.generations):
#             #print(f"Generazione {gen}, miglior costo: {best_cost}, costo medio: {np.mean([cost for cromo, cost in population])}")
#             next_gen = []
#             for _ in range(self.offprint//2):
#                 parents = []
#                 for _ in range(2):
#                     candidates = random.sample(population, 2)
#                     parents.append(min(candidates, key=lambda x: x[1])[0])
#                 # per ogni iterazione 2 figli

#                 # estraggo un valore tra 0 e 1
#                 ratio = random.random()
#                 if ratio < 0.8:
#                     # mutation
#                     # print("MUTAZIONE")
#                     #print("Genitore 1:", parents[0])
#                     #print("Genitore 2:", parents[1])
#                     offspring1=self._mutate_merge_tours(parents[0])
#                     offspring2=self._mutate_merge_tours(parents[1])
#                 else:
#                     # crossover
#                     #print("CROSSOVER")
#                     offspring1 = self._crossover(parents[0], parents[1])
#                     offspring2 = self._crossover(parents[1], parents[0])
                
#                 next_gen.extend([offspring1, offspring2])

            
#             next_gen = [(ind, cost) for ind, cost in sorted(next_gen, key=lambda x: x[1])]  
#             population = next_gen[:self.pop_size]  # mantieni solo i migliori
        
#             if population[0][1] < best_cost:
#                 best_chromo, best_cost = population[0]
#                 stagnation_counter = 0
#             else:
#                 stagnation_counter += 1
#             # se il costo rimane lo stess per 5 generazioni, interrompiamo
#             #if stagnation_counter >= 5:
#              #  break

#         return best_chromo
    
#    # unire due tours
#     def _merge_two_tours(self, tour_A, tour_B) -> tuple[list[tuple[int, float]], float]:

#         city_gold_map = {}
#         for c, w in tour_A + tour_B:
#             if w > 0:
#                 city_gold_map[c] = w
        
#         if not city_gold_map:
#             joined_tours= [(0,0)]+ tour_A+ [(0,0)]+ tour_B + [(0,0)]
#             return joined_tours, self._path_cost(joined_tours)
        
#         target_cities = list(city_gold_map.keys())
        
#         # --- LOGICA TSP STRATEGICO ---
#         merged_tour = []
        
#         # 1. Troviamo la città nel cluster che è più LONTANA dal deposito
#         far_city = max(target_cities, key=lambda c: self.dist_matrix[0][c])
        
#         # ma assicuriamoci che l'ordine tenda a "chiudere" verso il deposito.
#         current_node = far_city
        
#         merged_tour.extend(self.all_paths[0][current_node][:-1])  # percorso dal deposito alla città più lontana
#         target_cities.remove(current_node)
         
#         while target_cities:
#             # Tra le rimaste, scegliamo la più vicina alla posizione attuale
#             next_node = min(target_cities, key=lambda c: self.dist_matrix[current_node][c])
#             merged_tour.extend(self.all_paths[current_node][next_node][:-1])  # percorso tra current_node e next_node
#             target_cities.remove(next_node)
#             current_node = next_node
        
#         merged_tour.extend(self.all_paths[current_node][0])  # percorso dalla città finale al deposito

#         merged_tour_with_weights = [ ]
#         for c in merged_tour:
#             w= city_gold_map[c] if c in city_gold_map else 0
#             merged_tour_with_weights.append((c,w))
        
#         cost_merged = self._path_cost( merged_tour_with_weights)   
        
#         return merged_tour_with_weights, cost_merged

#    # starting from baseline and try to unificate the tours
#     def improved_baseline_ind (self):

#         # ordinare le città in base alla vicinanza
#         current_city=0
#         cities_to_visit= self.cities_to_visit
#         target_cities=[]
#         while cities_to_visit:
#             next_city = min(cities_to_visit, key=lambda c: self.dist_matrix[current_city][c])
#             target_cities.append(next_city)  # percorso tra current_node e next_node
#             cities_to_visit.remove(next_city)
#             current_city = next_city
        
#         path= []
#         tours=[]
#         for target_city in target_cities:
#             tour=[]
#             if self.graph.nodes[target_city]['gold'] == 0:
#                 continue  # se non c'è oro, salto la città
#             andata= self.all_paths[0][target_city]  # percorso dal deposito alla città target
#             tour.extend([(c, 0) for c in andata[:-1]])  # aggiungo il percorso di andata (escludendo la città target)

#             w= self.graph.nodes[target_city]['gold']
#             tour.append((target_city, w))

#             ritorno= self.all_paths[target_city][0]  # percorso di ritorno al deposito
#             tour.extend([(c, 0) for c in ritorno[1:-1]])  # aggiungo il percorso di ritorno (escludendo city target e deposito)  
            
#             path.extend(tour)
#             #print(f"Tour found {tour}")
#             tours.append(tour[1:])
        
#         path.extend([(0, 0)])
#         #print(f"path: {path}")
#         cost= self._path_cost(path)  

#         #print(tours)
        
#         diff=1

#         while diff > 0 :
#             # se il costo continua a migliorare aggiorno i tour
#             diff=0
#             copy_tours= tours.copy()

#             print(f" Difference: {diff}")
#             i=0
#             while i <len(copy_tours)-1: 
#                 a=i
#                 b=i+1
#                 tour_A= copy_tours[a]
#                 tour_B= copy_tours[b]
#                 #print(f"Merge {tour_A}, {tour_B}")
#                 separate_cost= self._path_cost([(0,0)] +tour_A+ [(0,0)])+ self._path_cost([(0,0)]+tour_B+ [(0,0)])
#                 #print(f"Separate cost {separate_cost}")
#                 merged_tour, merged_cost= self._merge_two_tours(tour_A, tour_B)
#                 #print(f"Merged cost {merged_cost}")
#                 if merged_cost< separate_cost:
#                     copy_tours[a]= merged_tour[1:-1]
#                     copy_tours.pop(b)
#                     print(f"Merged tour cost:{merged_cost}, Merge tour: {merged_tour}")
#                 else: 
#                     i+=1
                    
#                 tours= copy_tours
#                 new_route = [(0, 0)]
#                 for tour in tours:
#                     new_route.extend(tour)
#                     new_route.append((0, 0))

#                 #print(f" new route {new_route}")
#                 new_cost= self._path_cost(new_route)
#                 diff= cost - new_cost
#                 cost= new_cost
                
#         path = [(0, 0)]
#         for tour in tours:
#             path.extend(tour)
#             path.append((0, 0))
#           # aggiungo il ritorno finale al deposito
        
#         return path, self._path_cost(path)
    

#     def solution(self):
#         best_chromo = self._run_ga_logic()
#         return best_chromo, self._path_cost(best_chromo)

In [53]:
from src.GA_solver import GA_Solver

In [ ]:

def compute_ga_params(n_cities, beta, alpha):
    """Dynamically compute GA parameters based on problem complexity."""
    
    # Compute complexity factor: considers number of cities, weight penalty, and cost factor
    complexity = np.log(n_cities + 1) * (1 + beta) * (1 + alpha / 10)
    
    if n_cities <= 50:
        pop_size = min(50, n_cities)
        generations = 100
        offprint = int(pop_size * 0.6)
    
    elif n_cities <= 100:
        pop_size = min(100, n_cities)
        generations = 80 if beta >= 2 else 100
        offprint = int(pop_size * (0.3 if beta >= 2 else 0.5))
    
    elif n_cities <= 200:
        pop_size = min(100, n_cities // 2)
        generations = 60 if beta >= 2 else 80
        offprint = int(pop_size * (0.25 if beta >= 2 else 0.4))
    
    else:  # n_cities > 200 (e.g., 1000)
        # For large problems, prioritize computational efficiency
        # Scale down population inverse to problem size
        base_pop = min(80, int(200 / np.sqrt(n_cities / 100)))
        
        if beta >= 2:
            # High beta: tours are split many times, very expensive
            pop_size = min(base_pop, int(50 + 30 / (1 + alpha)))
            generations = int(60 - 10 * beta / 5)
            offprint = max(5, int(pop_size * 0.15))
        else:
            # Low beta: can afford slightly more exploration
            pop_size = min(base_pop + 20, int(80 - 10 * beta))
            generations = int(80 - 5 * beta)
            offprint = max(8, int(pop_size * 0.3))
    
    # Safety bounds
    pop_size = max(10, min(pop_size, 150))
    generations = max(20, min(generations, 200))
    offprint = max(5, int(offprint))
    
    return pop_size, generations, offprint

In [ ]:

import importlib
from itertools import product
import Problem as ProblemModule
import src.GA_solver as GA_solver_module

importlib.reload(ProblemModule)
importlib.reload(GA_solver_module)

Problem = ProblemModule.Problem
GA_Solver = GA_solver_module.GA_Solver


cities_list = [50, 100, 200, 1000]
densities = [0.1, 0.2, 0.5, 1.0]
alphas = [0.1, 1, 2, 5]
betas = [0.1, 1, 2, 5]

results = []

for n_cities, density, alpha, beta in product(cities_list, densities, alphas, betas):
    p = Problem(n_cities, density=density, alpha=alpha, beta=beta)

    # when beta is high >=2 we need few generations, take wy more time to generate the population
    if beta >= 2:
        population_size = min(max(1, n_cities// beta), 200)
        generations = population_size*2
        offprint = int(population_size * 0.2 )
    else:
        population_size = min(n_cities, 200)
        generations = population_size * 2
        offprint = int(population_size * 0.5)

    solver = GA_Solver(p, pop_size=population_size, generations=generations, offprint=offprint)
    best_path, best_cost = solver.solution(fast=True)
    baseline_cost = p.baseline()
    print(
        f"{n_cities:4d} | d={density:.1f} | "
        f"a={alpha:.1f} | b={beta:.1f} | "
        f"rel={best_cost / baseline_cost * 100:.2f}% |"
        f"baseline={baseline_cost:.2f} | ga={best_cost:.2f} | "
        f"pop_size={population_size} | gen={generations} | off={offprint}"
    ) 
    results.append({
        "n_cities": n_cities,
        "density": density,
        "alpha": alpha,
        "beta": beta,
        "baseline_cost": baseline_cost,
        "ga_cost": best_cost,
        "relative_cost": best_cost / baseline_cost * 100
    })


  50 | d=0.1 | a=0.1 | b=0.1 | rel=42.78% |baseline=266.51 | ga=114.03 | pop_size=50 | gen=100 | off=25
  50 | d=0.1 | a=0.1 | b=1.0 | rel=99.23% |baseline=2280.92 | ga=2263.38 | pop_size=50 | gen=100 | off=25
  50 | d=0.1 | a=0.1 | b=2.0 | rel=10.26% |baseline=52132.42 | ga=5350.07 | pop_size=25 | gen=50 | off=5
  50 | d=0.1 | a=0.1 | b=5.0 | rel=0.00% |baseline=2205641360.87 | ga=48594.05 | pop_size=10 | gen=20 | off=2
  50 | d=0.1 | a=1.0 | b=0.1 | rel=46.12% |baseline=311.41 | ga=143.62 | pop_size=50 | gen=100 | off=25
  50 | d=0.1 | a=1.0 | b=1.0 | rel=99.92% |baseline=21971.20 | ga=21953.39 | pop_size=50 | gen=100 | off=25
  50 | d=0.1 | a=1.0 | b=2.0 | rel=4.95% |baseline=5204023.63 | ga=257790.88 | pop_size=25 | gen=50 | off=5
  50 | d=0.1 | a=1.0 | b=5.0 | rel=0.00% |baseline=220564126776120.94 | ga=2589166091.72 | pop_size=10 | gen=20 | off=2
  50 | d=0.1 | a=2.0 | b=0.1 | rel=46.42% |baseline=327.08 | ga=151.82 | pop_size=50 | gen=100 | off=25
  50 | d=0.1 | a=2.0 | b=1.0 | 

In [114]:
from Problem import Problem

logging.getLogger().setLevel(logging.WARNING)

test_cases = [
    # 1. Il caso "Standard" (Equilibrio)
    Problem(100, density=0.4, alpha=1.0, beta=1.0),
    
    # 2. Il "Corriere Espresso" (Peso quasi irrilevante, vince chi fa meno km)
    Problem(100, density=0.6, alpha=0.05, beta=1.0),
    
    # 3. Il "Trasporto Eccezionale" (Peso punitivo, vince chi torna spesso al deposito)
    Problem(100, density=0.5, alpha=10.0, beta=1.0),
    
    # 4. L'incubo non-lineare (Costo esplosivo col peso)
    Problem(100, density=0.3, alpha=1.0, beta=2.0),
    
    # 5. Grafo Labirintico (Pochi collegamenti, obbliga a percorsi tortuosi)
    Problem(100, density=0.05, alpha=0.5, beta=1.0),
    
    # 6. Il caso "Super-Alpha" (Quasi ogni città richiede un viaggio dedicato)
    Problem(100, density=0.8, alpha=100.0, beta=1.0)
]

for p in test_cases:
    print(f"\n\n--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
    print(f"Baseline Cost: {p.baseline()}")

    solver = GA_Solver(p, pop_size=100, generations=50, offprint=50)
    best_path, best_cost = solver.solution()
    print(f"\n\nGA Cost: {best_cost}")
    print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")



--- PROBLEM: 100 cities, alpha 1.0, beta 1.0 ---
Baseline Cost: 19219.60601422488


GA Cost: 19213.528092778346
Relative cost: 99.97


--- PROBLEM: 100 cities, alpha 0.05, beta 1.0 ---
Baseline Cost: 1003.0302102994061


GA Cost: 983.4187191978162
Relative cost: 98.04


--- PROBLEM: 100 cities, alpha 10.0, beta 1.0 ---
Baseline Cost: 187421.74617630543


GA Cost: 187417.55101039688
Relative cost: 100.00


--- PROBLEM: 100 cities, alpha 1.0, beta 2.0 ---
Baseline Cost: 4403745.419646082


GA Cost: 183878.2466520997
Relative cost: 4.18


--- PROBLEM: 100 cities, alpha 0.5, beta 1.0 ---
Baseline Cost: 30641.331843608394


GA Cost: 30593.891978169635
Relative cost: 99.85


--- PROBLEM: 100 cities, alpha 100.0, beta 1.0 ---
Baseline Cost: 1822566.7309310008


GA Cost: 1822566.730931001
Relative cost: 100.00


In [50]:
from Problem import Problem
p=Problem(100, density=0.2, alpha=0.05, beta=1)
solver= GA_Solver(p, pop_size=10, generations=200, offprint=50)
best_path, best_cost = solver._create_baseline_individual()
print("Baseline path:", best_path)
print("Baseline cost:", best_cost)
best_path, best_cost = solver._create_nearest_neighbor_individual()
print("Nearest Neighbor path:", best_path)
print("Nearest Neighbor cost:", best_cost)
print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")

best_path, best_cost = solver.improved_baseline()
print("Improved baseline path:", best_path)
print("Improved baseline cost:", best_cost)
print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")


Baseline path: [(0, 0), (1, 777.6274334029364), (0, 0), (38, 0), (54, 0), (2, 971.8545996349064), (54, 0), (38, 0), (0, 0), (3, 501.24044501614), (0, 0), (38, 0), (4, 144.75360504869954), (38, 0), (0, 0), (3, 0), (5, 14.922351420493342), (3, 0), (0, 0), (1, 0), (27, 0), (6, 230.4263739688564), (27, 0), (1, 0), (0, 0), (53, 0), (72, 0), (55, 0), (7, 132.69039556873452), (55, 0), (72, 0), (53, 0), (0, 0), (53, 0), (72, 0), (55, 0), (8, 677.9810149392446), (55, 0), (72, 0), (53, 0), (0, 0), (40, 0), (9, 122.7106721239026), (40, 0), (0, 0), (53, 0), (33, 0), (10, 506.8236016890124), (33, 0), (53, 0), (0, 0), (11, 694.5681732072436), (0, 0), (36, 0), (12, 581.5354926116814), (36, 0), (0, 0), (53, 0), (72, 0), (13, 200.57587600839756), (72, 0), (53, 0), (0, 0), (38, 0), (4, 0), (18, 0), (14, 804.3204016560804), (18, 0), (4, 0), (38, 0), (0, 0), (40, 0), (79, 0), (15, 715.6917224861859), (79, 0), (40, 0), (0, 0), (53, 0), (72, 0), (55, 0), (7, 0), (16, 739.2450199116262), (7, 0), (55, 0), (72

In [48]:
def check_path(problem, path):
    total_cost = 0
    current_weight = 0
    is_correct=True
    total_gold= sum(problem.graph.nodes[ n ]['gold'] for n in problem.graph.nodes if n !=0 )

    for i in range(len(path)-1):
        u, v = path[i][0], path[i+1][0]
        d= problem.graph[u][v]['dist']
        total_cost += d + (problem.alpha * d * current_weight) ** problem.beta
        # Aggiorna il peso se si raccoglie oro
        if v != 0:  # Non raccogliere oro al deposito
            current_weight += path[i+1][1]
            total_gold -= path[i+1][1]
                
        else:
            current_weight = 0  # Svuota il peso al deposito
    is_correct = abs(total_gold) < 1e-6  # Tolleranza per errori float
    
    return is_correct, total_cost

is_correct, calculated_cost = check_path(p, best_path)
print(f"Calcolated Cost from path: {calculated_cost}")
print(f"Path is correct: {is_correct}")

Calcolated Cost from path: 1354.8904985474137
Path is correct: True
